In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
   !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
#from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import sequence
tfds.disable_progress_bar()

# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
train_filepath = "train-data.tsv"
test_filepath = "valid-data.tsv"

train_df = pd.read_csv(train_filepath, sep="\t", header=None, names=["type", "msg"])
train_df.dropna() #Remove rows with missing values
train_df.head() #Display the first 5 data

# Convert and read tsv into a Data Frame
# sep="\t": Indicates that the values in the file are separated by tabs (\t).
# header=None: Specifies that the file doesn't have a header row.
# names=["type", "msg"]: Assigns column names "type" and "msg" to the DataFrame.

In [ ]:
test_df = pd.read_csv(test_filepath, sep="\t", header=None, names=["type", "msg"])
test_df.dropna()
test_df.head()

In [ ]:
train_df["type"] = pd.factorize(train_df["type"])[0]
test_df["type"] = pd.factorize(test_df["type"])[0]
train_df.head()

# COnverts the Text Labels into Numerical Labels: the labels in 'type' column which are 'spam' and 'ham' are numercially respresented(1 and 0, respectfully)
# pd.factorize(): This function takes a sequence of values (like the 'type' column) and converts them into unique numerical codes. It returns two arrays: one with the numerical codes and another with the unique values that were encoded.

In [ ]:
train_labels =  train_df["type"].values #Selecting and extracting 'type' column in DataFrame into train_labels; .values() makes the data compatible to tensorflow
train_ds = tf.data.Dataset.from_tensor_slices( #takes slices of the input data and creates individual elements for the dataset.
    (train_df["msg"].values, train_labels) #This line creates a TensorFlow Dataset object called train_ds using the message text and corresponding labels
)

In [ ]:
test_labels =  test_df["type"].values
test_ds = tf.data.Dataset.from_tensor_slices(
    (test_df["msg"].values, test_labels)
)
test_ds.element_spec

In [ ]:
BUFFER_SIZE = 100 #Used for shuffling and training; larger buffer allows better randomization but consumes more memory
BATCH_SIZE = 32
train_ds = train_ds.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

#Shuffling: shuffles/randomizes the training data to improve model generalization
#Batches: Divides the shuffled data into batches of specified size(efficient processing)
#Prefetching:  prefetches data for the next batch while the current batch is being processed, minimizing idle time and speeding up training.

In [ ]:
vec = TextVectorization( # A tool specifically designed to convert text into numbers.
    output_mode='int', #Unique int
    max_tokens=1000,
    output_sequence_length=1000,
)

vec.adapt(train_ds.map(lambda text, label: text))#extracts the text data from your train_ds dataset, ignoring the labels

# Where the TextVectorization layer learns from your training data.
# The TextVectorization layer analyzes the text and builds a vocabulary of the most frequent words. It learns how to map each word to a unique integer.

In [ ]:
vocab = np.array(vec.get_vocabulary()) # Retrieves the vocabulary that the TextVectorization layer has built
vocab[:20]

In [ ]:
len(vec.get_vocabulary())

In [ ]:
model = tf.keras.Sequential([ #Neural Network MOdel
    vec,#layer that turns text into unique numerical data
    tf.keras.layers.Embedding(#Transforms numerical representation into dense vectors that represent the meaning(similiar words have vectors that are closer)
        len(vec.get_vocabulary()),
        64, # Dimensionality of the dense vectors
        mask_zero=False,
    ),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)), #Processes the sequence of word embeddings in forward and backward directions, capturing contextual info
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(1000, activation='relu'), #1000 Neurons, ReLU activation func
    tf.keras.layers.Dense(500, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3), #30% dropout of neurons
    tf.keras.layers.Dense(1) #Produced score/probability for classification ('spam' or 'ham')
])


model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(1e-4),
    metrics=['accuracy'],
)

In [ ]:
history = model.fit(
    train_ds,
    validation_data=test_ds,
    validation_steps=30,
    epochs=10,
)
# The model's performance is evaluated on the validation dataset after each epoch.
# This helps monitor how well the model is generalizing to unseen data during training.

In [ ]:
test_loss, test_acc = model.evaluate(test_ds)

In [ ]:
print('Test Loss:', test_loss)
print('Test Accuracy' , test_acc)

In [ ]:
plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.xlabel("Epochs")
plt.ylabel('accuracy')
plt.legend(['accuracy', 'val_accuracy'])
plt.ylim(None, 1)

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel("Epochs")
plt.ylabel('loss')
plt.legend(['loss', 'val_loss'])
plt.ylim(0, None)

In [ ]:
h = history.history
print(h['loss'])
print(h['val_loss'])
print(h['accuracy'])
print(h['val_accuracy'])

#Loss, Val_Loss, Accuracy, Val_Accuracy over the last 10 epoches

In [ ]:
# function to predict messages based on model
# (Returning prediction and label, ex. [0.008318834938108921, 'ham'])

def predict_message(pred_text):
  # # Convert pred_text to a tf.data.Dataset
  pred_text_ds = tf.data.Dataset.from_tensor_slices([pred_text])
  # # Adapt the TextVectorization layer to the new dataset
  pred_text_ds = pred_text_ds.batch(1).prefetch(tf.data.AUTOTUNE)
  pred = model.predict(pred_text_ds)
  print(pred)
  prob = tf.sigmoid(pred[0][0]).numpy() #Sigmoid function "squashes" the logit value into range of 0 to 1, making it a probability instead
  return [prob, "ham" if prob <0.5 else "spam"]

pred_text = "Sale today! to stop texts call 98912460324" #"how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
